In [1]:
import sys
sys.path.append('../src')

In [2]:
from covid import daily_snapshot
from pathlib import Path

import population as pop
import pandas as pd
import requests

In [3]:
pd.set_option('max_rows', 100) 

In [4]:
date = input('date in mm-dd-yyyy format')

date in mm-dd-yyyy format 04-02-2020


In [5]:
state = input('state, e.g. New Jersey')

state, e.g. New Jersey New York


### Quick glance at the data

In [6]:
df = daily_snapshot(date)
df.head()

,Unnamed: 0,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
FIPS,,,,,,,,,,,,
45001.0,NaN,Abbeville,South Carolina,US,2020-04-02 23:25:27,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
22001.0,NaN,Acadia,Louisiana,US,2020-04-02 23:25:27,30.295065,-92.414197,61,1,0,0,"Acadia, Louisiana, US"
51001.0,NaN,Accomack,Virginia,US,2020-04-02 23:25:27,37.767072,-75.632346,10,0,0,0,"Accomack, Virginia, US"
16001.0,NaN,Ada,Idaho,US,2020-04-02 23:25:27,43.452658,-116.241552,312,3,0,0,"Ada, Idaho, US"
19001.0,NaN,Adair,Iowa,US,2020-04-02 23:25:27,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [7]:
df[df.Country_Region=='US'].describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
count,0.0,2263.000000,2263.000000,2312.000000,2312.000000,2312.000000,2312.0
mean,NaN,37.901839,-90.296335,105.299740,2.563149,3.893166,0.0
std,NaN,5.056066,14.162025,1181.910573,33.492062,187.196122,0.0
min,NaN,13.444300,-159.596679,0.000000,0.000000,0.000000,0.0
25%,NaN,34.226043,-95.801391,2.000000,0.000000,0.000000,0.0
50%,NaN,38.006710,-88.110438,6.000000,0.000000,0.000000,0.0
75%,NaN,41.459743,-82.330657,24.250000,1.000000,0.000000,0.0
max,NaN,64.807262,145.673900,51809.000000,1397.000000,9001.000000,0.0


### Aggregate by state

In [8]:
state_totals = df[df.Country_Region=='US'].groupby(by='Province_State').sum()

In [9]:
state_totals.head()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,0.0,2076.132667,-5465.814957,1233,32,0,0
Alaska,0.0,362.539360,-862.001024,143,3,0,0
Arizona,0.0,505.138555,-1671.948482,1715,32,0,0
Arkansas,0.0,1917.467085,-5085.094606,643,12,0,0
California,0.0,1873.911674,-6028.510306,10773,238,0,0


In [10]:
state_totals['death_rate?'] = state_totals['Deaths'] / state_totals['Confirmed'] * 100.0

In [11]:
state_totals.describe()

,Unnamed: 0,Lat,Long_,Confirmed,Deaths,Recovered,Active,death_rate?
count,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.0,57.000000
mean,0.0,1478.825184,-3523.113909,4197.465517,102.172414,155.189655,0.0,2.387042
std,0.0,1218.832299,3044.446499,12540.143413,321.174497,1181.889202,0.0,2.197478
min,0.0,0.000000,-14256.489929,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,398.189159,-5458.201885,316.000000,6.000000,0.000000,0.0,1.544402
50%,0.0,1233.777309,-3210.628444,853.500000,21.000000,0.000000,0.0,2.098476
75%,0.0,2469.858479,-881.128482,3003.750000,77.750000,0.000000,0.0,2.597403
max,0.0,4665.624024,145.673900,92506.000000,2373.000000,9001.000000,0.0,16.666667


In [12]:
state_covid = df[df.Province_State == state].copy()
state_covid['death_rate'] = 100 * state_covid['Deaths'] / state_covid['Confirmed']
state_columns=['Admin2', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'death_rate' ]

In [13]:
sort_by = input('sort by one of Admin2, Confirmed, Deaths, death_rate')

sort by one of Admin2, Confirmed, Deaths, death_rate Confirmed


In [14]:
state_covid[state_columns].sort_values(by=sort_by, ascending=False)

,Admin2,Confirmed,Deaths,Recovered,Active,death_rate
FIPS,,,,,,
36061.0,New York City,51809,1397,0,0,2.696443
36119.0,Westchester,11567,64,0,0,0.553298
36059.0,Nassau,10587,76,0,0,0.717862
36103.0,Suffolk,8746,69,0,0,0.788932
36087.0,Rockland,3751,18,0,0,0.479872
36071.0,Orange,1993,19,0,0,0.953337
36027.0,Dutchess,667,5,0,0,0.749625
36029.0,Erie,617,12,0,0,1.944895
36055.0,Monroe,420,9,0,0,2.142857


## We need to locate county data to add per capita rates.